In [7]:
!pip install spotipy
!pip install geopandas
!pip install geopy
!pip install requests
!pip install lxml
!pip install pandas
!pip install numpy
!pip install gmplot

You should consider upgrading via the '/Users/annie/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 994 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 11.5 MB/s eta 0:00:01     |████████████████████████▋       | 11.4 MB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 8.8 MB/s eta 0:00:01▍       | 778 kB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 3.0 MB/s  eta 0:00:01
You should consider upgrading via the '/Users/annie/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 112 kB 4.0 MB/s eta 0:00:01
You should consider upgrading via the '/Users/annie/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/annie/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consi

In [8]:
import requests
import spotipy
import geopandas
import geopy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
from lxml import html
import re
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim

## Spotify Scraper

### Connect to Spotify API

In [9]:
client_id = '693faf0bce4f4fb0aa5fc5d8b3cc0b25'
client_secret = 'c2ab38a172f44774a0cfae1d7734130f'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

### Scraper Functions
get_artistID_by_artistName - check for reliability, better search

In [10]:
#not deemed reliable yet
def get_artistID_by_artistName(artistName):
        artist = sp.search(q=artistName, type="artist")
        #print(artist)
        artistObject = artist['artists']['items']
        if len(artistObject) == 0:
            return "NOT IN SPOTIFY"
        else:
            artistID = artistObject[0]['id']
            #print('AID: ', artistID)
            return artistID;

def get_genre(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain genre
    genres = artist['genres']
    if len(genres) == 0:
        return 'NONE'
    else:
        return genres

def get_followers(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain follower count
    followers = int(artist['followers']['total'])
    return followers

def get_monthlyListeners(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Monthly Listeners', r.text)
    print(r.text)
    #get monthly listeners starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split(',')
    monthlyListeners = strings1[0].split(",")
    #grab second string
    return int(monthlyListeners[0].lstrip())

def get_cities(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Where People Listen', r.text)
    #get where people listen starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split('"')
    cities = strings1[0].split(",")
    #grab second string
    return cities

## Songkick Scraper

### Connect to Songkick API

In [11]:
# Define API Key
api_key = "FYSeGSNlg6HXIfAr"

### Scraper Functions

get_venueArtists: add in festival/lineup capabilities

get_venueGenres: clean up mapping

get_venueSize: Decide how to use followers/listeners, fix metric (within the 30-60 percentile?), regression using Songkick Venue Capacity?

In [12]:
venue_latLong_info = pd.DataFrame()
def get_venueArtists(metroID):
    global venue_latLong_info
    venue_latLong_info = pd.DataFrame(columns = ['Venue', 'Latitude', 'Longitude'])

    # request json
    r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ str(metroID) +"/calendar.json?apikey=" + str(api_key))
    # parse for events
    events = r.json()["resultsPage"]["results"]["event"]
    # define dictionary for storage
    venue_artist_dict = {}
    #loop through events
    for i in range(len(events)):
        # The artist
        if events[i]["performance"] != []:
            artist = events[i]["performance"][0]["artist"]["displayName"]
        else:
            artist = events[i]["displayName"]
        # Venue Name
        venue = events[i]["venue"]["displayName"]
        venueLat = events[i]["venue"]["lat"]
        venueLng = events[i]["venue"]["lng"]

       # venueLat = events[i]["venue"]["lat"]
        # venueLong = events[i]["venue"]["lng"]
        if(venueLat == None):
                venueLat = None
        else:
                venue_latLong_info = venue_latLong_info.append({'Venue':str(venue),'Latitude':venueLat,'Longitude':venueLng}, ignore_index=True)
        # if the venue is not already in dict, add it with the artist as its value
        if venue not in venue_artist_dict.keys():
            venue_artist_dict[venue] = [artist]
        # if it is in the dict, add the act to its 
        else:
            if artist not in venue_artist_dict[venue]:
                venue_artist_dict[venue].append(artist)
    return venue_artist_dict

def get_venueGenres(venue_artist_dict):
    venue_genre_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        genre_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, ignore it
            if id_return == 'NOT IN SPOTIFY':
                artist_genre = 'NONE'
                # get the genre of artists in spotify
            else:
                artist_genre = get_genre(id_return)
                print
                mapped_genre=-1
                # replace the genre with a code... NEED TO CLEAN UP
                if any("funk" in s for s in artist_genre):
                    mapped_genre = 0
                if any("rock" in s for s in artist_genre):
                    mapped_genre = 1 
                if any("punk" in s for s in artist_genre):
                    mapped_genre = 1 
                if any("alternative" in s for s in artist_genre):
                    mapped_genre = 1
                if any("alt" in s for s in artist_genre):
                    mapped_genre = 1
                if any("reggae" in s for s in artist_genre):
                    mapped_genre = 2
                if any("r&b" in s for s in artist_genre):
                    mapped_genre = 3
                if any("hip hop" in s for s in artist_genre):
                    mapped_genre = 3
                if any("soul" in s for s in artist_genre):
                    mapped_genre = 3
                if any("rap" in s for s in artist_genre):
                    mapped_genre = 4
                if any("bounce" in s for s in artist_genre):
                    mapped_genre = 4
                if any("pop" in s for s in artist_genre):
                    mapped_genre = 5
                if any("jazz" in s for s in artist_genre):
                    mapped_genre = 6
                if any("blues" in s for s in artist_genre):
                    mapped_genre = 6
                if any("folk" in s for s in artist_genre):
                    mapped_genre = 6
                if any("electronic" in s for s in artist_genre):
                    mapped_genre = 7
                if any("house" in s for s in artist_genre):
                    mapped_genre = 7
                if any("indie" in s for s in artist_genre):
                    mapped_genre = 8
                if any("country" in s for s in artist_genre):
                    mapped_genre = 9
                if any("cajun" in s for s in artist_genre):
                    mapped_genre = 9
              
                # add genre to list
                genre_list.append(mapped_genre)
        # assign genres to venue
        #venue_genre_dict[venue] = list(set([x for x in genre_list if x != -1]))
        venue_genre_dict[venue] = list(filter(lambda x: x != -1, genre_list))

    # return dictionary
    return venue_genre_dict


def get_venueSize(venue_artist_dict):
    venue_size_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        size_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, its size is basically 0
            if id_return == 'NOT IN SPOTIFY':
                size_list.append(0)
            # get their followers
            else:
                artist_followers = get_followers(id_return)
                size_list.append(artist_followers)
        venue_size_dict[venue] = np.mean(np.array(size_list))
    return venue_size_dict

In [13]:
#get requested city id
#example


cityIDs = []
def get_city_id(locations):
    cityIDs = []
    for i in range(len(locations)):
        r = requests.get("https://api.songkick.com/api/3.0/search/locations.json?query=" + locations[i] + "&apikey=" + str(api_key))
        cityID = r.json()["resultsPage"]["results"]["location"][0]["metroArea"]["id"]
        cityIDs.append(cityID)
    return cityIDs


## Matching Functions
Both: Need to figure out a way to clean it up


In [22]:
#
# FUNCTION TO ADD ELEMENTS TO DICT
#
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], tuple):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

#
# GENRE MATCH FUNCTION
# if artist genre identity is a match for venue genre, match (hard constraints)
#

def genre_match(d, genre_of):
    #print('in genre check function')
    genre_filtered_dictionary = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in d.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass

    return genre_filtered_dictionary

def size_match(d,size_of):
    perfect_match_dict={}
    for key,value in d.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(percentage1>=.4):
            perfect_match_dict[key]= [round(percentage1, 2),num]
            None
    return perfect_match_dict

def rank_match(city_venue_genres,city_venue_sizes, genre_of, size_of):
    from collections import Counter
    
    print(num_genre)
    genre_filtered_dictionary = {}
    genre_percents = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in city_venue_genres.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass
    print(genre_filtered_dictionary)
    num_genre = dict.fromkeys(genre_filtered_dictionary)
    
    for key, value in genre_filtered_dictionary.items():
        total_value = 0
        total_num_genres = 0
        for j in value:
            total_value += 1
            total_num_genres += num_genre[j]
        append_value(genre_percents, key, 1/(total_value/total_num_genres))
    
    perfect_match_dict={}
    for key,value in city_venue_sizes.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(percentage1>=.4):
            perfect_match_dict[key]= round(percentage1, 2)
            None
    
    totals = dict.fromkeys(genre_percents)
    for key in totals:
        totals[key] = perfect_match_dict.get(key)*2 + genre_percents.get(key)
    
    temp = max(totals.values())
    result = [key for key in percents if percents[key] == temp]

    return result

## New Orleans Example

In [15]:
#use names from class... THIS PART WILL BE CHANGED!

names = ["Vampire Weekend","The Doobie Brothers", "Greta Van Fleet", "Tank and the Bangas", "Men I Trust"]
def create_df(names):
    df = pd.DataFrame({"band_name": names})
    df["id"] = df.band_name.apply(get_artistID_by_artistName)
    df["genre"] = df.id.apply(get_genre)
    #df["listeners"] = df.id.apply(get_monthlyListeners)
    df["followers"] = df.id.apply(get_followers)
    #pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth',None)
    df['mapped_genre'] = [[] for _ in range(df.shape[0])]

    for i in range(df.genre.size):
        if any("funk" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('0')

        if any("rock" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')

        elif any("punk" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')
            
        elif any("alternative" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')
            
        elif any("alt" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')

        if any("reggae" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('2')

        if any("r&b" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('3')
            
        elif any("hip hop" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('3')

        if any("rap" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('4')

        if any("pop" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('5')

        if any("jazz" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('6')
            
        elif any("blues" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('6')
            
        elif any("folk" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('6')

        if any("electronic" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('7')
            
        elif any("house" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('7')

        if any("indie" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('8')

        if any("country" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('9')
            
        elif any("cajun" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('9')
            
    return df
df = create_df(names)

In [14]:
#
# OUTPUT SECTION
# change this portion when looking at different bands
#
cityID = "11772"
def output_matching_df(cityID, df):
    city = get_venueArtists(cityID)
    city_venue_genres = get_venueGenres(city)
    city_venue_sizes = get_venueSize(city)
    column_names = ["Artist", "Artist Genre", "Artist Size", "Genre Matches", "Size Matches", "Set"]
    output_df = pd.DataFrame(columns = column_names)

    for i in range(0,len(df)):
        #get artist genres
        artist_genre = []
        for j in range(0,len(df.mapped_genre[i])):
            artist_genre.append(int(df.mapped_genre[i][j]))

        #get artist size
        artist_size = df.followers[i]

        #calculate genre matches
        genre_matches = genre_match(city_venue_genres ,artist_genre)


        #calculate size matches
        size_matches = size_match(city_venue_sizes ,artist_size)
        output_df = output_df.append({'Artist':df.band_name[i],'Artist Genre':artist_genre,
                                      'Artist Size':artist_size,'Genre Matches':genre_matches,
                                      'Size Matches':size_matches}, ignore_index=True)

    return output_df
#output_matching_df(cityID, df)

def output_matching_df_rank(cityID, df):
    city = get_venueArtists(cityID)
    city_venue_genres = get_venueGenres(city)
    city_venue_sizes = get_venueSize(city)
    column_names = ["Artist", "Artist Genre", "Artist Size", "Genre Matches", "Size Matches", "Set"]
    output_df = pd.DataFrame(columns = column_names)

    for i in range(0,len(df)):
        #get artist genres
        artist_genre = []
        for j in range(0,len(df.mapped_genre[i])):
            artist_genre.append(int(df.mapped_genre[i][j]))

        #get artist size
        artist_size = df.followers[i]

        #calculate genre matches
        venue_one = rank_match(city_venue_genres,city_venue_sizes, artist_genre, artist_size)
        output_df = output_df.append({'Artist':df.band_name[i],'Artist Genre':artist_genre,
                                      'Artist Size':artist_size,'Ranked Match':venue_one}, ignore_index=True)

    return output_df
output_matching_df_rank(cityID, df)

,Artist,Artist Genre,Artist Size,Genre Matches,Size Matches,Set
0,Vampire Weekend,"[1, 5, 8]",1709401,"{'Southport Hall': [5, 4], 'Gasa Gasa': [[[8, ...",{},NaN
1,The Doobie Brothers,"[1, 6, 9]",1522935,"{'The Howlin' Wolf': [[6, 6, 0], [6, 6, 0]], '...",{},NaN
2,Greta Van Fleet,"[1, 6]",1404622,"{'The Howlin' Wolf': [[6, 6, 0], [6, 6, 0]]}",{},NaN
3,Tank and the Bangas,"[0, 8]",160609,"{'The Howlin' Wolf': [6, 6, 0], 'Shrine on Air...","{'Tipitina's': [0.64, 103134.0]}",NaN
4,Men I Trust,"[5, 8]",486520,"{'Southport Hall': [5, 4], 'Gasa Gasa': [[[8, ...","{'Southport Hall': [0.62, 303150.0]}",NaN


## Traveling Salesperson Problem/Algorithm (TSP) - Example with Tank and the Bangas

### Use python-tsp library functions for TSP

In [18]:
!pip install python-tsp
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import great_circle_distance_matrix

## Tank and The Bangas tour in New Orleans

In [19]:
locations = ["New Orleans, LA"]
names = ["Tank and the Bangas"]

In [20]:
def oneCityTour(locations, names):
    cities = get_city_id(locations)
    df = create_df(names)
    for cityID in cities:
        print(cityID)
        arr = []
        MatchesList = []
        genreDict = output_matching_df(cityID, df).iloc[0]['Genre Matches']
        for key in genreDict.keys():
            MatchesList.append(key)
            venueLat = venue_latLong_info.loc[venue_latLong_info['Venue'] == key, 'Latitude'].values
            venueLng = venue_latLong_info.loc[venue_latLong_info['Venue'] == key, 'Longitude'].values
            venueLat = venueLat[0]
            venueLng = venueLng[0]
            arr.append([venueLat, venueLng])
        #following functions are from python-tsp library
        sources = np.array(arr)
        distance_matrix = great_circle_distance_matrix(sources)
        permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
        distance = distance/1609
        print(distance)
        permutation
        s = pd.Series(permutation)
        mappingDict = {}
        for i in range(len(s)):
            mappingDict[i] = MatchesList[i]
        s = s.map(mappingDict)
        optimalPath = s.tolist()
        print(optimalPath)

oneCityTour(["New Orleans, LA"],["Tank and the Bangas"])

11772
19.055186430386005
["The Howlin' Wolf", 'Unknown venue', 'Broadside', 'Shrine on Airline (Zephyr Field)', 'Gasa Gasa', "Tipitina's"]


## Multiple cities tour for Vampire Weekend

In [21]:
names = ["Vampire Weekend"]
locations = ["Chicago, IL", "Los Angeles, CA", "Boston, MA", "Dallas, TX"]

In [39]:
##NEED TO TEST GENRE MATCHES HERE
arr = []
bestMatchesList = []

def multipleCityTours(names, locations):
    df = create_df(names)
    cities = get_city_id(locations)
    cities
    for cityID in cities:

        sizeDict = output_matching_df(cityID, df).iloc[0]['Size Matches']
        bestMatch = max(sizeDict, key=sizeDict.get)
        bestMatchesList.append(bestMatch)
        venueLat = venue_latLong_info.loc[venue_latLong_info['Venue'] == bestMatch, 'Latitude'].values
        venueLng = venue_latLong_info.loc[venue_latLong_info['Venue'] == bestMatch, 'Longitude'].values
        venueLat = venueLat[0]
        venueLng = venueLng[0]
        arr.append([venueLat, venueLng])
    sources = np.array(arr)
    distance_matrix = great_circle_distance_matrix(sources)
    permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
    distance = distance/1609
    print(distance)
    permutation
    s = pd.Series(permutation)
    mappingDict = {}
    for i in range(len(s)):
        mappingDict[i] = bestMatchesList[i]
    s = s.map(mappingDict)
    optimalPath = s.tolist()
    print(optimalPath)


multipleCityTours(names, locations)

5381.122932306415
['Rosemont Theatre', 'Mayan Theater', 'Rodeo West', 'Royale']


## Test Cases

In [184]:
def check_genreMatches(locations, names):
    cities = get_city_id(locations)
    df = create_df(names)
    venue_info = []
    for cityID in cities:
        arr = []
        MatchesList = []
        sizeDict = output_matching_df(cityID, df).iloc[0]['Size Matches']
        genreDict = output_matching_df(cityID, df).iloc[0]['Genre Matches']
        genreMatches = []
        sizeMatches =[]
        for key in genreDict:
            genreMatches.append(key)
        for key in sizeDict:
            sizeMatches.append(key)
        setList = [value for value in genreMatches if value in sizeMatches]
        if(setList != None):
            for set1 in setList:
                venue_info.append(set1)
                venue_info.append(sizeDict.get(set1))
                venue_info.append(genreDict.get(set1))
            return venue_info
        else:
            if sizeDict != {}:
                bestMatch = max(sizeDict, key=sizeDict.get)
                venue_size = sizeDict.get(bestMatch)
                venue_genre = genreDict.get(bestMatch)
            else:
                bestMatch = None
            MatchesList.append(bestMatch) 
            MatchesList.append(venue_size)
            MatchesList.append(venue_genre)
        
            return MatchesList
check_genreMatches(["Houston, TX"],["Blacktop Mojo"])

['Scout Bar', [0.93, 46361.0], [[1, 4, 1], [1, 4, 1]]]

In [187]:
def get_venue_info(locations, venue):
    cityID = get_city_id(locations)
    city = get_venueArtists(cityID[0])
    city_venue_genres = get_venueGenres(city)
    city_venue_sizes = get_venueSize(city)
    city_venue_genres = get_venueGenres(city)
    city_venue_sizes = get_venueSize(city)
    valuesList = []

    if venue in city_venue_sizes:
        value = city_venue_sizes[venue]
    else:
        value = ''
        
    if venue in city_venue_genres:
        value1 = city_venue_genres[venue]
    else:
        value1 = ''
    valuesList.append(value)
    valuesList.append(value1)
    return valuesList 
get_venue_info(["Houston, TX"], "The Studio, Warehouse Live")

[5970.0, []]

In [194]:
import csv
with open('Tour Comparison-Capstone Tour Planner - Testing CSV.csv','r') as csvinput:
    with open('output3.csv', 'w') as csvoutput:
        writer = csv.writer(csvoutput)
        line_count = 0

        for row in csv.reader(csvinput):
            if line_count == 0:
                writer.writerow(row)
                line_count += 1
            else:      
                value = get_venue_info([row[2]],row[3])
                value1 = check_genreMatches([row[2]],[row[0]])
                writer.writerow([row[0]] +[row[1]] + [row[2]] + [[row[3]]+[value]] + [value1])
                print(line_count)
                line_count += 1


1
2
3
4
5
6
7
8
9
10
